## Init

In [2]:
# from urllib.parse import quote
import pandas as pd
import os
import sys
import importlib
import glob
import json

# my packages
from dData.dData import dData

# utilities
plot = pd.DataFrame
from itertools import combinations

from bs4 import BeautifulSoup

# import all module in order to use reload later on the paclet
import importlib
import toolsGeneral.main
import toolsOSM.overpass
import tools.filters

# use this line to ensure intellisense recognizes new symbols
from toolsGeneral.main import *
from toolsOSM.overpass import *
from tools.filters import *

def reloadPckg():
    importlib.reload(toolsGeneral.main)
    importlib.reload(tools.filters)
    globals().update({
        name: getattr(toolsGeneral.main, name) 
        for name 
        in dir(toolsGeneral.main) 
        if not name.startswith('_')
    })
    # global dump
    # dump = toolsGeneral.main.dump



In [3]:
osmCountriesMeta = load(r"C:\Users\gonta\D\code\development\OSM\get countries ids and add labels\data\osmCountriesAddMetaInfo.json")
osmCountriesIdsDict = load(r"c:\Users\gonta\D\code\development\OSM\get countries ids and add labels\data\osmCountriesIdsDict.json")

In [4]:
cntrName = "Canada"
cntrRaw = f"./data/{osmCountriesIdsDict[cntrName]}/rawOSMRes.json"
cntrDataDir = f"./data/{osmCountriesIdsDict[cntrName]}/"
cntrId = osmCountriesIdsDict[cntrName]
cntrId

'1428125'

# collect

## make overpass query and dump

In [5]:
rawOSMRes = getOSMAddsTRecursedown(cntrId, [4,6,8])
list(rawOSMRes.keys())

['version', 'generator', 'osm3s', 'elements']

In [6]:
dump(cntrRaw, rawOSMRes)

# sanitize

In [52]:
rawOSMRes = load(cntrRaw)
print(list(rawOSMRes.keys()))

['version', 'generator', 'osm3s', 'elements']


In [72]:
testSanitize(rawOSMRes)

missing name:  0
duplicate elements:  6


[[], [9271165, 9117919, 9107934, 11722184, 9334290, 17542341]]

In [65]:
import copy
rawOSMResTSane = copy.deepcopy(rawOSMRes)
print(len(rawOSMResTSane['elements']))
rawOSMResTSane['elements'] = [ele for ele in rawOSMRes['elements'] if not (ele['tags'].get('parent_id', '') == '1354835' and ele['id'] == 9271165)]
rawOSMResTSane['elements'] = [ele for ele in rawOSMResTSane['elements'] if not (ele['tags'].get('parent_id', '') == '9344588' and ele['id'] == 9117919)]
rawOSMResTSane['elements'] = [ele for ele in rawOSMResTSane['elements'] if not (ele['tags'].get('parent_id', '') == '11681120' and ele['id'] == 9107934)]
rawOSMResTSane['elements'] = [ele for ele in rawOSMResTSane['elements'] if not (ele['tags'].get('parent_id', '') == '9123338' and ele['id'] == 11722184)]
rawOSMResTSane['elements'] = [ele for ele in rawOSMResTSane['elements'] if not (ele['tags'].get('parent_id', '') == '9082149' and ele['id'] == 9334290)]
rawOSMResTSane['elements'] = [ele for ele in rawOSMResTSane['elements'] if not (ele['tags'].get('parent_id', '') == '12375630' and ele['id'] == 17542341)]
len(rawOSMResTSane['elements'])

1512


1506

In [73]:
testSanitize(rawOSMResTSane)

missing name:  0
duplicate elements:  0


[[], []]

# normalize

#### make relsDataIndex

In [74]:
relsDataIndex = {str(x["id"]):x for x in rawOSMResTSane["elements"]}
# add parent_id to keys on first level
for id, value in relsDataIndex.items():
    if value["tags"]["admin_level"] == '2':
        value["parent_id"] = "#"
    elif value["tags"]["admin_level"] == osmCountriesMeta[cntrName]["addLvlsNum"][0]:
        value["parent_id"] = cntrId
    elif value["tags"]["admin_level"] in osmCountriesMeta[cntrName]["addLvlsNum"][1:]:
        value["parent_id"] = value["tags"]["parent_id"]      
        # del value["parent_id"]

    value["id"] = str(id)
    # del value["id"]
    relsDataIndex[id] = sortDictKeys(value, ["type", "id", "parent_id", "tags"])

len(list(relsDataIndex.keys()))

1506

#### make childsIndex

In [75]:
idPairs = {key:value["parent_id"] for key, value in relsDataIndex.items() if value["parent_id"] != "#"} 
# creates an empty dictionary
childsIndex = {}
for k, v in idPairs.items():
    # if key doesn't exist then make list else append to list
    childsIndex.setdefault(v, list()).append(str(k))
    
len(childsIndex)

64

In [76]:
dump(os.path.join(cntrDataDir, "childsIndex.json"), childsIndex)

In [77]:
# Add childs to rels data index
for id, value in relsDataIndex.items():
    relsDataIndex[id] = sortDictKeys({"childs":childsIndex.get(id, []),**relsDataIndex[id]}, ["type", "id", "parent_id", "childs","tags"])

In [78]:
dump(os.path.join(cntrDataDir, "relsDataIndex.json"), relsDataIndex)

#### [OLD] make data tree and min tree 

In [26]:
# make tags properties to use in selection
relTagsIndex = { k : v["tags"] for k, v in relsDataIndex.items()}

In [ ]:
indexKeys = list(childsIndex.keys())

def makeAddsTree(idList, dataIndex):
    return {id: {**dataIndex[id], "childs":(makeAddsTree(relsDataIndex[id]["childs"], dataIndex) if id in indexKeys else {})} for id in idList }

In [29]:
keysMin = ["name"]
keys = ["id", "name", "official_name", "parent_id", "ISO3166-2", "admin_level"]
addsDataTree = makeAddsTree([countryID], {k:dictFilterKeys(v, keysMin) for k, v in relTagsIndex.items()})
len(addsDataTree[countryID]["childs"])

17

In [30]:
dump(os.path.join(outDir, "addsDataTree.json"), addsDataTree)

# validate

In [79]:
relsDataIndex = load(os.path.join(cntrDataDir, "relsDataIndex.json"))
print(len(relsDataIndex))
childsIndex = load(os.path.join(cntrDataDir, "childsIndex.json"))
print(len(childsIndex))

1506
64


In [80]:
print(len(relsDataIndex))
# 1. Check leak of divisions from other countries
test('leaksFromOtherCountries')(relsDataIndex, cntrId)
# 2. Intersection between childs
print('Intersection between childs?: ', len(findIntersection(list(childsIndex.values()))))

1506
Entities that not belong to the country:  0
Intersection between childs?:  0


## review

In [81]:
relsDataIndex = load(os.path.join(cntrDataDir, "relsDataIndex.json"))
print(len(relsDataIndex))
childsIndex = load(os.path.join(cntrDataDir, "childsIndex.json"))
print(len(childsIndex))

1506
64


In [82]:
lvl1 = [v["id"] for v in relsDataIndex.values() if v["tags"]["admin_level"] == '4']
print(len(lvl1))
lvl2 = [v["id"] for v in relsDataIndex.values() if v["tags"]["admin_level"] == '6']
print(len(lvl2))
lvl3 = [v["id"] for v in relsDataIndex.values() if v["tags"]["admin_level"] == '8']
print(len(lvl3))

13
101
1391


In [83]:
getKeys = ["id", "name", "official_name", "parent_id", "ISO3166-2", "admin_level","childs_num"]

### lvl 1

In [84]:
props = [{**dictRemoveKeys(relsDataIndex[id],["childs", "tags"]), **relsDataIndex[id]["tags"], "childs_num":len(relsDataIndex[id]["childs"])} for id in lvl1]
props = [{k:t.get(k, "missing") for k in getKeys} for t in props]
plot(props)

,id,name,official_name,parent_id,ISO3166-2,admin_level,childs_num
0,61549,Québec,missing,1428125,CA-QC,4,0
1,68841,Ontario,missing,1428125,CA-ON,4,0
2,68942,New Brunswick / Nouveau-Brunswick,New Brunswick / Nouveau-Brunswick,1428125,CA-NB,4,0
3,390558,Nova Scotia,missing,1428125,CA-NS,4,74
4,390840,ᓄᓇᕗᑦ Nunavut,missing,1428125,CA-NU,4,0
5,390841,Manitoba,missing,1428125,CA-MB,4,0
6,390867,British Columbia,missing,1428125,CA-BC,4,0
7,391115,Prince Edward Island,missing,1428125,CA-PE,4,27
8,391178,Saskatchewan,missing,1428125,CA-SK,4,0
9,391186,Alberta,missing,1428125,CA-AB,4,0


### lvl 2

In [87]:
props = [{**dictRemoveKeys(relsDataIndex[id],["childs", "tags"]), **relsDataIndex[id]["tags"], "childs_num":len(relsDataIndex[id]["childs"])} for id in relsDataIndex["390558"]["childs"]]
props = [{k:t.get(k, "missing") for k in getKeys} for t in props]
plot(props)

,id,name,official_name,parent_id,ISO3166-2,admin_level,childs_num
0,1306290,Municipality of the District of Lunenburg,missing,390558,missing,6,0
1,1306380,Chester District Municipality,missing,390558,missing,6,0
2,1306765,Region of Queens Municipality,missing,390558,missing,6,0
3,1307096,Municipality of the District of Shelburne,missing,390558,missing,6,52
4,1307160,Municipality of the District of Barrington,missing,390558,missing,6,44
...,...,...,...,...,...,...,...
69,17542441,Eskasoni First Nation,missing,390558,missing,6,2
70,17543377,Acadia First Nation,missing,390558,missing,6,3
71,17543391,Acadia First Nation,missing,390558,missing,6,1
72,17546215,Pictou Landing First Nation,missing,390558,missing,6,0


### lvl 3

In [88]:
props = [{**dictRemoveKeys(relsDataIndex[id],["childs", "tags"]), **relsDataIndex[id]["tags"], "childs_num":len(relsDataIndex[id]["childs"])} for id in relsDataIndex["1307096"]["childs"]]
props = [{k:t.get(k, "missing") for k in getKeys} for t in props]
plot(props)

,id,name,official_name,parent_id,ISO3166-2,admin_level,childs_num
0,8467962,Jordan Falls,missing,1307096,missing,8,0
1,8651898,Welshtown,missing,1307096,missing,8,0
2,8652513,Lower Ohio,missing,1307096,missing,8,0
3,9150571,Little Port L'Hebert,missing,1307096,missing,8,0
4,9150595,Louis Head,missing,1307096,missing,8,0
5,9150596,West Middle Sable,missing,1307096,missing,8,0
6,9150597,East Sable River,missing,1307096,missing,8,0
7,9150742,Western Head,missing,1307096,missing,8,0
8,9150755,Little Harbour,missing,1307096,missing,8,0
9,9150756,East Side of Ragged Island,missing,1307096,missing,8,0


## Join add tree files

### [OLD] make data tree

In [12]:
base = list(map(lambda x: load()(x), glob.glob("data/*/addsDataMinTree.json")))

In [ ]:
# addsDataTree = makeAddsTree([countryID], {k:dictFilterKeys(v, keysMin) for k, v in relTagsIndex.items()})
# len(addsDataTree[countryID]["childs"])

In [76]:
dump(os.path.join("data", "addsDataMinJoinedTree.json"), addsDataMinJoinedTree)

### [Dev]

In [11]:
ids = ['2','3']
index = {'2':['a','b'],3:['c'],'a':['d']}
dataIndex = {'2':{'tags':{'name':'name-2'}},
             '3':{'tags':{'name':'name-3'}},
             'a':{'tags':{'name':'name-a'}},
             'b':{'tags':{'name':'name-b'}},
             'c':{'tags':{'name':'name-c'}},
             'd':{'tags':{'name':'name-d'}}}

In [ ]:
htmlTree = makeHTMLTree(['2'], index, dataIndex)
print(htmlTree)

### [OLD] make tree for each country

In [15]:
htmlTree = makeHTMLTree([cntrId], childsIndex, relsDataIndex)
# print(htmlTree)

In [16]:
import re
htmlTree = re.sub(r'^<ul>', '', htmlTree)
htmlTree = re.sub(r'</ul>$', '', htmlTree)

In [109]:
soup = BeautifulSoup(htmlTree, 'html.parser')
pretty_html = soup.prettify()
dump(treeOutputDir, pretty_html)

In [69]:
importlib.reload(toolsGeneral.main)
importlib.reload(toolsOSM.overpass)
from toolsGeneral.main import (load, dump)
from toolsOSM.overpass import (makeTree, makeHTMLTree)

### make countries trees and join

In [5]:
countrs = list(filter(lambda x: re.match(r'\d+', x), os.listdir("./data")))
print(countrs)

['1428125', '167454', '2202162', '288247']


In [6]:
idChildsDataIndex = list(map(lambda x: [x,load(f"./data/{x}/childsIndex.json"), load(f"./data/{x}/relsDataIndex.json")], countrs))

In [12]:
reloadPckg()

In [7]:
countrsTrees = list(map(lambda x: makeHTMLTree([x[0]], x[1], x[2]), idChildsDataIndex))

In [14]:
dir = {'1':'a', '2': 'b'}
value = next((dir[key] for key in ['3','2'] if key in dir), None)
print(value)

b


In [104]:
next((dir[key] for key in ['4','2'] if key in dir))

'b'

In [8]:
def formattify(tree):
    formatted = re.sub(r'^<ul>', '', tree)
    formatted = re.sub(r'</ul>$', '', formatted)
    soup = BeautifulSoup(formatted, 'html.parser')
    formatted = soup.prettify()
    return formatted

formattedTree = "<div id=\"addSelectionTree\"><ul>" + ''.join(list(map(formattify, countrsTrees))) + "</ul></div>"

### insert tree into html page

In [9]:
soup = BeautifulSoup(load(r"c:\Users\gonta\D\study sync\full stack\projects\administrative divisions query\data\indexBase.html"), "html.parser")

In [10]:
forReplace = BeautifulSoup(formattedTree, 'html.parser')
toReplace  = soup.find(id="addSelectionTree")
toReplace.replace_with(forReplace)

<div id="addSelectionTree">
<!-- insert html tree here -->
</div>

In [11]:
dump('./data/index.html', str(soup))